<a href="https://colab.research.google.com/github/CHOIHANJOON/python/blob/main/5_3_%ED%8A%B8%EB%A6%AC%EC%9D%98_%EC%95%99%EC%83%81%EB%B8%94.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
#부트 스트랩 : 데이터세트에서 중복을 허용하여 뎅이터를 샘플링 하는 방식
# randomForestClassifier클래스를 화이트 와인을 분류하는 문제에 적용해보기
# 먼저 와인 데이터셋을 판다스로 불러오고 훈련세트와 테스트 세트로 나눕니다.


In [3]:
import numpy as np
import pandas as pd
from sklearn.model_selection import train_test_split
wine = pd.read_csv('https://bit.ly/wine_csv_data')
data = wine[['alcohol', 'sugar', 'pH']].to_numpy()
target = wine['class'].to_numpy()
train_input, test_input, train_target, test_target = train_test_split(data, target, test_size=0.2, random_state=42)

In [6]:
# n_jons 는 씨피유 코어 갯수 지정 하는 것
# return_train_scire 를 True로 지정하면 검증 점수뿐 아니라 훈련세트에 대한 점수도 같이 반환합니다.
# 훈련 세트와 검증세트의 점수를 비교하면 과대적합을 파악하는데 용이함 (기본 매겨변수는 False임)

from sklearn.model_selection import cross_validate
from sklearn.ensemble import RandomForestClassifier
rf = RandomForestClassifier(n_jobs=-1, random_state=42)
scores = cross_validate(rf, train_input,train_target,
                        return_train_score=True, n_jobs=-1)
print(np.mean(scores['train_score']), np.mean(scores['test_score']))

0.9973541965122431 0.8905151032797809


In [7]:
#랜덤 포레스트의 특성 중요도는 각 결정 트리의 특성 중요도를 취합한것.
#랜덤 포레스트 모델을 훈련 세트에 훈련한 후 특성 중유도 출력
rf.fit(train_input, train_target)
print(rf.feature_importances_)

[0.23167441 0.50039841 0.26792718]


In [9]:
#oob_score 함수를 사용하여 각 결정 트리의 OOB 점수를 평균하여 출력합니다.
rf = RandomForestClassifier(oob_score=True, n_jobs=-1, random_state=42)
rf.fit(train_input, train_target)
print(rf.oob_score_)

0.8934000384837406


In [10]:
# ExtraTree 를 사용하여 교차 검증점수 확인
from sklearn.ensemble import ExtraTreesClassifier
et = ExtraTreesClassifier(n_jobs=-1, random_state=42)
scores = cross_validate(et, train_input, train_target,
                        return_train_score=True, n_jobs=-1)
print(np.mean(scores['train_score']), np.mean(scores['test_score']))

0.9974503966084433 0.8887848893166506


In [11]:
et.fit(train_input, train_target)
print(et.feature_importances_)

[0.20183568 0.52242907 0.27573525]


In [12]:
#히스토그램 기반 그레이디언트 부스팅 은 정형데이터를 다루는 머시러닝 알고리즘 중 최고인기 알고리즘
#HistGrandientBoostingClassifier 는 먼저 입력 특성을 256 구간으로 나눠 최적의 분할된 노드를 매우 빠르게 찾을 수 있음
# 256개의 구간 중에서 하나를 떼어 놓고 누락된 값을 위해서 사용. ?
#HistGrandientBoostingClassifier 를 와인 데이터셋ㅅ에 적용해보자~

In [15]:
from sklearn.experimental import enable_hist_gradient_boosting
from sklearn.ensemble import HistGradientBoostingClassifier
hgb = HistGradientBoostingClassifier(random_state=42)
scores = cross_validate(hgb, train_input, train_target,
                        return_train_score=True)
print(np.mean(scores['train_score']), np.mean(scores['test_score']))

0.9321723946453317 0.8801241948619236


In [16]:
# 특성 중요도를 계산하기 위해 permutation_importance() 함수를 사용 하여 특성을 하나씩 랜덤하게 섞어서 모델의 성능이
# 변화하는지를 관찰하여 어떤 특성이 중요한지를 계산한다.
# n_repeats / 랜덤하게 섞을 횟수를 지정 10으로 지정, 기본값은 5
from sklearn.inspection import permutation_importance

hgb.fit(train_input, train_target)
result=permutation_importance(hgb, train_input, train_target,
                              n_repeats=10, random_state=42, n_jobs=-1)
print(result.importances_mean)

[0.08876275 0.23438522 0.08027708]


In [17]:
#permutation_importance() 함수가 반환하는 객체는 반복하여 얻은 특성 중요도(importances),평균(importances_mean),표준 편차(importances_std)
#을 담고있다.
# 테스트 세트에서 특성 중요도를 계산 하여 봅시다~

result = permutation_importance(hgb, test_input, test_target, n_repeats=10, random_state=42, n_jobs=-1)
print(result.importances_mean)

[0.05969231 0.20238462 0.049     ]


In [18]:
# HistGradientBoostingClassifier를 사용해 테스트 세트에서의 성능을 최종적으로 확인해 보자

hgb.score(test_input, test_target)

0.8723076923076923

In [22]:
#XGBoost = 그레이디언트 부스팅 알고리즘을 구현한 라이브러리
#XGBoost를 사용해 와인데이터의 교차 검증 점수를 확인해봅시당

from xgboost import XGBClassifier
xgb = XGBClassifier(tree_method='hist', random_state=42)
scores=cross_validate(xgb,train_input, train_target,
                      return_train_score=True)
print(np.mean(scores['train_score']), np.mean(scores['test_score']))

0.8824322471423747 0.8726214185237284


In [25]:
#LightGBM = 마이크로소프트에서 만든 히스토그램 기반 그레이디언트 부스팅 라이브러리

from lightgbm import LGBMClassifier
lgb = LGBMClassifier(random_state=42)
scores = cross_validate(lgb, train_input, train_target, return_train_score=True, n_jobs=-1)

# 5-3 트리의 앙상블 정리 ###

정형데이터 = 데이터 베이스에 저장되는 데이터 ex:CSV, DB, EXEL
비정형데이터 = DB나 엑셀로 표현하기 어려운 것들 텍스트데이터,사진,디지털음악,

앙상블 학습이란? : 결정트리를 기반으로 정형데이터를 다루는데 가장 뛰어난 성과를 내는 알고리즘

랜덤포레스트이란? : 앙상블 학습의 대표 주자 중 하나로 안정적 / 결정트리를 랜덤하게 만들어 숲을 만들고 이로 인해 각 결정 트리의 예측을 사용해 최종 예측을 만듬
부트스트랩 샘플이란? : 데이터 세트에서 중복을 허용하여 데이터를 샘플링 하는 방식

엑스트라 트리란? : 기본적으로 100개의 결정트리를 훈련을 함 랜덤 포레스트와 동일하게 결정트리가 제공하는 대부분의 매개변수를 지원.
전체 특성 중에 일부 특성을 랜덤하게 선택하여 노드를 분할하는데 사용.
단 랜덤포레스트와의 차이점은 부트스트랩 샘플을 사용하지 않는다.

그레이디언트 부스팅 이란? : 깊이가 얕은 결정 트리를 사용하여 이전 트리의 오차를 보완하는 방식으로 앙상블 하는 방법.
경사하강법을 사용하여 트리를 앙상블에 추가함.
분류에서는 로지스틱 손실 함수를 사용하고, 회귀에서는 평균 제곱 오차 함수를 사용.

히스토그램 기반 그레이디언트 부스팅 이란? : 정형데이터를 다루는 머신러닝 알고리즘 중에서 가장 인기가 많은 알고리즘,
입력특성을 256개의 구간으로 나누어 노드를 분할할 때 최적의 분할을 매우 빠르게 찾을 수 있음.